<a href="https://colab.research.google.com/github/AMEYABHAVE/Cristl/blob/main/HAR_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
print(tf.__version__)

2.6.0


In [2]:
from google.colab import drive
import pandas as pd
file_location = '/content/drive/MyDrive/accelerometer.csv'

In [3]:
accelerometer =pd.read_csv(file_location)
accelerometer.drop(columns=['time','repetition','uuid'], inplace=True)
accsitting= accelerometer[accelerometer['activity'] == 'CHAIR'].head(9875)
accstanding= accelerometer[accelerometer['activity'] == 'STANDING'].head(9875)
acclying= accelerometer[accelerometer['activity'] == 'LYING_UP'].head(9875)
accdataset = pd.DataFrame()
accdataset = accdataset.append([accsitting, accstanding, acclying])
accdataset.drop(columns=['activity'], inplace=True)

file_location1 = '/content/drive/MyDrive/gyroscope.csv'
gyroscope =pd.read_csv(file_location1)
gyroscope.drop(columns=['time','repetition','uuid'], inplace=True)
gyrsitting= gyroscope[gyroscope['activity'] == 'CHAIR'].head(9875)
gyrstanding= gyroscope[gyroscope['activity'] == 'STANDING'].head(9875)
gyrlying= gyroscope[gyroscope['activity'] == 'LYING_UP'].head(9875)
gyrdataset = pd.DataFrame()
gyrdataset = gyrdataset.append([gyrsitting, gyrstanding, gyrlying])

dataset=pd.concat([accdataset, gyrdataset.set_index(accdataset.index)], axis=1)
dataset.drop(columns=['name'], inplace=True)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29625 entries, 0 to 93591
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   acc_x     29625 non-null  float64
 1   acc_y     29625 non-null  float64
 2   acc_z     29625 non-null  float64
 3   gyr_x     29625 non-null  float64
 4   gyr_y     29625 non-null  float64
 5   gyr_z     29625 non-null  float64
 6   activity  29625 non-null  object 
dtypes: float64(6), object(1)
memory usage: 1.8+ MB


In [5]:
dataset['activity'].value_counts()

LYING_UP    9875
CHAIR       9875
STANDING    9875
Name: activity, dtype: int64

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
print(dataset.head())

      acc_x     acc_y     acc_z     gyr_x     gyr_y     gyr_z activity
0 -0.414825  7.590469  6.235336 -0.309540  7.679596  6.142029    CHAIR
1 -0.337662  7.645508  6.125275 -0.530884  7.897354  5.765747    CHAIR
2 -0.393295  7.605423  6.231155 -0.608643  8.231140  5.266846    CHAIR
3 -0.469269  7.592865  6.281403 -0.570953  8.260452  5.229172    CHAIR
4 -0.418411  7.633530  6.317307 -0.816223  8.244308  5.284195    CHAIR


In [7]:
X = dataset[['acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z']]
Y = dataset['activity']
scaler = StandardScaler()
X = scaler.fit_transform(X)
scaled_X = pd.DataFrame(data=X, columns=['acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z'])
scaled_X['activity'] = Y.values
scaled_X

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,activity
0,-0.610501,0.705138,0.431830,-0.511514,0.743974,0.380241,CHAIR
1,-0.525900,0.725603,0.415590,-0.751855,0.819761,0.323345,CHAIR
2,-0.586896,0.710698,0.431213,-0.836288,0.935931,0.247909,CHAIR
3,-0.670191,0.706029,0.438628,-0.795364,0.946133,0.242213,CHAIR
4,-0.614432,0.721149,0.443926,-1.061684,0.940514,0.250533,CHAIR
...,...,...,...,...,...,...,...
29620,1.077678,1.494336,-0.388644,0.393923,1.434373,-0.432256,LYING_UP
29621,1.107858,1.472311,-0.378226,0.417320,1.446039,-0.434066,LYING_UP
29622,1.135394,1.471199,-0.370548,0.326331,1.445205,-0.435876,LYING_UP
29623,1.105884,1.486325,-0.393499,0.383524,1.451038,-0.441668,LYING_UP


In [8]:
X_train = scaled_X[scaled_X.columns[:6]]
Y_train = scaled_X[scaled_X.columns[6:7]]

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_train)

In [10]:
def train_test_split(X, y, split_size=0.8):
    split= int(len(X) * split_size)
    train_x = X[:split]
    train_y = y[:split]
    test_x = X[split:]
    test_y = y[split: ]
    return train_x, test_x, train_y, test_y

X_train,X_test,Y_train, Y_test =train_test_split(X_train, Y_train)

print("X_train shape ", X_train.shape)
print("Y train shape ", Y_train.shape)
print("X_test shape ", X_test.shape)
print("y_test shape ", Y_test.shape)

X_train shape  (23700, 6)
Y train shape  (23700,)
X_test shape  (5925, 6)
y_test shape  (5925,)


In [11]:
from keras.preprocessing.sequence import TimeseriesGenerator

time_steps = 100
features = 6

train_gen=TimeseriesGenerator(X_train.to_numpy(), Y_train, length=time_steps, batch_size=1024)
test_gen=TimeseriesGenerator(X_test.to_numpy(), Y_test, length=time_steps, batch_size=1024)

In [12]:
model=Sequential()
model.add(LSTM(32, return_sequences=True, input_shape = (time_steps, features),
              kernel_regularizer = l2(0.000001), bias_regularizer = l2(0.000001), name='lstm_1'))  model.add(Flatten(name='flatten')) 
model.add(Dense(64, activation='relu',kernel_regularizer = l2(0.000001), bias_regularizer = l2(0.000001), name='dense_1' )) 
model.add(Dense(len(np.unique(Y_train)), activation='softmax', 
                kernel_regularizer = l2(0.000001), bias_regularizer = l2(0.000001), name='output'))
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 32)           4992      
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                204864    
_________________________________________________________________
output (Dense)               (None, 3)                 195       
Total params: 210,051
Trainable params: 210,051
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [14]:
from keras.callbacks import ModelCheckpoint
callbacks= [ModelCheckpoint('model.h5', save_weights_only=False, save_best_only=True, verbose=1)]

In [15]:
history = model.fit(train_gen, epochs=5, validation_data=test_gen, callbacks=callbacks)

Epoch 1/5
24/24 [==============================] - 10s 338ms/step - loss: 0.9728 - accuracy: 0.5283 - val_loss: 1.2896 - val_accuracy: 0.7674

Epoch 00001: val_loss improved from inf to 1.28958, saving model to model.h5
Epoch 2/5
24/24 [==============================] - 8s 314ms/step - loss: 0.5443 - accuracy: 0.7783 - val_loss: 1.6739 - val_accuracy: 0.7674

Epoch 00002: val_loss did not improve from 1.28958
Epoch 3/5
24/24 [==============================] - 7s 312ms/step - loss: 0.4062 - accuracy: 0.8416 - val_loss: 1.1289 - val_accuracy: 0.7676

Epoch 00003: val_loss improved from 1.28958 to 1.12886, saving model to model.h5
Epoch 4/5
24/24 [==============================] - 8s 312ms/step - loss: 0.3628 - accuracy: 0.8362 - val_loss: 1.4230 - val_accuracy: 0.7610

Epoch 00004: val_loss did not improve from 1.12886
Epoch 5/5
24/24 [==============================] - 8s 313ms/step - loss: 0.3503 - accuracy: 0.8589 - val_loss: 1.5345 - val_accuracy: 0.7612

Epoch 00005: val_loss did not

In [16]:
from keras.models import load_model
model=load_model('model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 32)           4992      
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                204864    
_________________________________________________________________
output (Dense)               (None, 3)                 195       
Total params: 210,051
Trainable params: 210,051
Non-trainable params: 0
_________________________________________________________________
